### Passo 1 - Importar Arquivos e Bibliotecas

In [1]:
import pandas as pd
import win32com.client as win32
import pathlib
import os

### Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [2]:
emails = pd.read_excel(r'Bases de dados\Emails.xlsx')
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')
lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', sep=';', encoding='latin1')


### Passo 3 - Salvar a planilha na pasta de backup

In [3]:
#incluir loja no vendas

vendas = vendas.merge(lojas, on= 'ID Loja')

In [4]:
dicionario_lojas = {}

for loja in lojas['Loja']:
    dicionario_lojas[loja] = vendas.loc[vendas['Loja'] == loja,:]

In [7]:
dia_indicador = vendas['Data'].max()

In [8]:
caminho_backup = pathlib.Path(r"Backup Arquivos Lojas")

arquivos_pasta_backup = caminho_backup.iterdir() ##verifica itens dentro da pasta
lista_nomes_backup = [arquivo.name for arquivo in arquivos_pasta_backup]

for loja in dicionario_lojas:
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()
        
    nome_arquivo = '{}_{}_{}.xlsx'.format(dia_indicador.month,dia_indicador.day, loja)
    local_arquivo = caminho_backup / loja / nome_arquivo
    dicionario_lojas[loja].to_excel(local_arquivo)

### Passo 4 - Calcular o indicador para 1 loja

In [23]:
loja = 'Norte Shopping'
vendas_loja = dicionario_lojas[loja]
vendas_loja_dia = vendas.loc[vendas['Data'] == dia_indicador, :]

#faturamento
faturamento_ano = vendas_loja['Valor Final'].sum()
print(faturamento_ano)
faturamento_dia = vendas_loja_dia['Valor Final'].sum()
print(faturamento_dia)

#diversidade produtoos

qtde_produtos_ano = len(vendas_loja['Produto'].unique())
print(qtde_produtos_ano)

qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())
print(qtde_produtos_dia)


#ticket medio an

valor_venda = vendas_loja.groupby('Código Venda').sum()
ticket_medio_ano = valor_venda['Valor Final'].mean()
print(ticket_medio_ano)


valor_venda = vendas_loja_dia.groupby('Código Venda').sum()
ticket_medio_dia = vendas_loja_dia['Valor Final'].mean()
print(ticket_medio_dia)

1711968
21591
120
35
784.5866177818515
553.6153846153846


C:\Users\franc\AppData\Local\Temp\ipykernel_15000\1958962488.py:22: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  valor_venda = vendas_loja.groupby('Código Venda').sum()
C:\Users\franc\AppData\Local\Temp\ipykernel_15000\1958962488.py:27: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  valor_venda = vendas_loja_dia.groupby('Código Venda').sum()


### Passo 5 - Enviar por e-mail para o gerente

### Passo 6 - Automatizar todas as lojas

### Passo 7 - Criar ranking para diretoria

### Passo 8 - Enviar e-mail para diretoria